# Document to Structured Data

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set Up Azure OpenAI

In [2]:
import os
import openai
from dotenv import load_dotenv

print('openai version: ', openai.__version__)

# Set up Azure OpenAI
load_dotenv()
openai.api_type = "azure"
openai.api_base = "https://cog-z6b6grosjkvfy.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "cdce6e9e7c774c44a42677efcdec8f38"


openai version:  0.27.8


False

## Deploy a Model

In [3]:
# id of desired_model
desired_model = 'gpt-4-32k' 
desired_capability = 'chat_completion' # apply as completion, since gpt-4 is only released as chat in Azure OpenAI

# list models deployed with
deployment_id = None
result = openai.Deployment.list()

for deployment in result.data:
    if deployment["status"] != "succeeded":
        continue
    
    model = openai.Model.retrieve(deployment["model"])
    print(model)
    # check if desired_model is deployed, and if it has 'completion' capability
    if model["id"] == desired_model and model['capabilities'][desired_capability]:
        deployment_id = deployment["id"]
        
# if no model deployed, deploy one
if not deployment_id:
    print('No deployment with status: succeeded found.')

    # Deploy the model
    print(f'Creating a new deployment with model: {desired_model}')
    result = openai.Deployment.create(model=desired_model, scale_settings={"scale_type":"standard"})
    deployment_id = result["id"]
    print(f'Successfully created {desired_model} that supports text {desired_capability} with id: {deployment_id}.')
else:
    print(f'Found a succeeded deployment of "{desired_model}" that supports text {desired_capability} with id: {deployment_id}.')

{
  "capabilities": {
    "fine_tune": false,
    "inference": true,
    "completion": true,
    "chat_completion": false,
    "embeddings": false,
    "scale_types": [
      "standard"
    ]
  },
  "lifecycle_status": "preview",
  "deprecation": {
    "inference": 1727654400
  },
  "id": "text-davinci-003",
  "status": "succeeded",
  "created_at": 1664496000,
  "updated_at": 1664496000,
  "object": "model"
}
{
  "capabilities": {
    "fine_tune": false,
    "inference": true,
    "completion": true,
    "chat_completion": true,
    "embeddings": false,
    "scale_types": [
      "standard"
    ]
  },
  "lifecycle_status": "generally-available",
  "deprecation": {
    "inference": 1696118400
  },
  "id": "gpt-35-turbo",
  "status": "succeeded",
  "created_at": 1678320000,
  "updated_at": 1678320000,
  "object": "model"
}
{
  "capabilities": {
    "fine_tune": false,
    "inference": true,
    "completion": false,
    "chat_completion": true,
    "embeddings": false,
    "scale_types": 

## Request API

In [4]:
response = openai.ChatCompletion.create(
  engine = deployment_id,
  messages = [{"role":"user","content":"who are you?"},],
  temperature=0.5,
  max_tokens=6000,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0, 
  stop=None)

response

<OpenAIObject chat.completion id=chatcmpl-7RRSIzkqcZNE3Ut0Ban5U8ol4u91Z at 0x1a8f8a409b0> JSON: {
  "id": "chatcmpl-7RRSIzkqcZNE3Ut0Ban5U8ol4u91Z",
  "object": "chat.completion",
  "created": 1686774750,
  "model": "gpt-4-32k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "I am an AI language model, and I am here to help answer your questions and have a conversation with you."
      }
    }
  ],
  "usage": {
    "completion_tokens": 23,
    "prompt_tokens": 11,
    "total_tokens": 34
  }
}

In [5]:
response['choices'][0]['message']['role']
response['choices'][0]['message']['content']

'assistant'

'I am an AI language model, and I am here to help answer your questions and have a conversation with you.'

In [6]:
def request_api(messages, deployment_id):
    response = openai.ChatCompletion.create(
        engine = deployment_id,
        messages = messages,
        temperature=0,
        max_tokens=6000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop='###')
    return response

## Get Structured Data

In [7]:
def get_structured_data(document, prompt_postfix, deployment_id):
    content = prompt_postfix.replace('<document>', document)
    messages = [{"role":"user","content":content},]; #print(messages)

    structured_data = request_api(messages, deployment_id)
    return structured_data

### Document Type: Resume

In [8]:
fname = "../data/resume.txt"
encodings = ['utf-8', 'latin-1', 'utf-16']

for encoding in encodings:
    try:
        with open(fname, 'r', encoding=encoding) as f:
            document = f.readlines()
        
        # convert list to str
        document = ' '.join(document)
        print(document)
        break  # Stop looping if the file is successfully read
    except UnicodeError:
        continue  # Move to the next encoding if there is a Unicode error


Contact
 chew.yean.yam@gmail.com
 www.linkedin.com/in/cyyam
 (LinkedIn)
 Top Skills
 Research
 Microarray Analysis
 OpenCV
 Languages
 English (Native or Bilingual)
 Malay (Professional Working)
 Mandarin (Native or Bilingual)
 Cantonese (Native or Bilingual)
 Certifications
 Uncovering Your Authentic Self at
 Work
 Worldwide Communities -
 Community SME 2018
 Fred Kofman on Managing Conflict
 Chew-Yean Yam
 AI Leader & Practitioner | Data Science | Machine Learning | AI
 Strategy
 United Kingdom
 Summary
 Over 20 years of industrial and applied research experience with
 global advanced technology companies and internationally renowned
 research institutions: Microsoft, Intel, HP Labs, Agency for Science
 Technology & Research, British Aerospace Engineering, National
 Physical Laboratory (UK).
 I build and lead Data Science and Machine Learning specialist teams
 to design and develop AI applications with start-ups and established
 enterprises. I advise on their AI strategies, identify 

In [9]:
# prompt
prompt_postfix = """ <document>
  \n###
  \nExtract the key sections from the resume above into json.
"""
structured_data = get_structured_data(document, prompt_postfix, deployment_id)

In [10]:
print(structured_data['choices'][0]['message']['content'])
print(type(structured_data['choices'][0]['message']['content']))


{
  "Contact": {
    "Email": "chew.yean.yam@gmail.com",
    "LinkedIn": "www.linkedin.com/in/cyyam"
  },
  "Top Skills": [
    "Research",
    "Microarray Analysis",
    "OpenCV"
  ],
  "Languages": {
    "English": "Native or Bilingual",
    "Malay": "Professional Working",
    "Mandarin": "Native or Bilingual",
    "Cantonese": "Native or Bilingual"
  },
  "Certifications": [
    "Uncovering Your Authentic Self at Work",
    "Worldwide Communities - Community SME 2018",
    "Fred Kofman on Managing Conflict"
  ],
  "Experience": {
    "Microsoft": {
      "Duration": "8 years 2 months",
      "Role": "AI Leader & Practitioner | Data Science | Machine Learning | AI Strategy"
    },
    "Bristol Data Science and Machine Learning Study Group": {
      "Location": "Bristol, United Kingdom",
      "Description": "Empower every single person in Bristol and surrounding areas to achieve quality Machine Learning and Data Science knowledge."
    },
    "Hewlett-Packard Laboratories": {
      

In [11]:
import json

output_string = structured_data['choices'][0]['message']['content']
output_dict = json.loads(output_string)



In [12]:
type(output_dict)

dict

In [13]:
with open('data.json', 'w') as file:
    # Write the dictionary to the JSON file
    json.dump(output_dict, file)

In [16]:
from flask import Flask, render_template
import csv

import pandas as pd
import json

with open('data.json') as file:
    data = json.load(file)

# creating an csv file
df = pd.json_normalize(data)
df.to_csv('data.csv', index=False)



In [17]:
app = Flask(__name__)

@app.route('/')
def index():
    data = []
    with open('data.csv', 'r') as file:
        csv_data = csv.DictReader(file)
        for row in csv_data:
            data.append(row)
    return render_template('index.html', data=data)


if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1